# Lab 3: Pandas Tutorial
## TA: Andrea Treviño Gavito
## Date: January 27th, 2020

In [ ]:
# Import libraries (run this block before anything else)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# inline plot
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5)

## Introduction to pandas

This lab is not meant to be a detailed demonstration of Pandas, but rather a tutorial on how to use pandas effectively for data analysis. We will not be going into the finer details. The tutorial and the exercises are, however, self-contained. If you are comfortable with Pandas, in particular with groupby and aggregate, you may skip the tutorial.

We will be analyzing the NYC-flights14 dataset in both the tutorial and the exercise. It contains flights data from the Bureau of Transporation Statistics for all the flights that departed from New York City airports from Jan-Oct 2014. The dataset is available at:

https://raw.githubusercontent.com/Rdatatable/data.table/master/vignettes/flights14.csv

Download it to the `data` folder either using the browser or using `wget` (available only on Linux/Mac) on the command line.

## Data Structures

There are two main data structures implemented in pandas:

1. `Series` is a one-dimensional labeled array capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.).
2. `DataFrame` is a 2-dimensional labeled data structure with columns of potentially different types. It is similar to R's data.frame. 

## Reading from delimiter-separated files

The `read_csv` function can be used to read delimiter-separated files. By default, it assumes that the file is comma-separated. We read in the flight14 dataset below.

In [ ]:
# load the flights14 dataset. Assumed to be in the data folder
# read_csv with the default options - first row header and comma-separated file
flights = pd.read_csv("../data/flights14.csv")
flights.head(10) # print first 10 rows

The column names can be accessed from the `.columns` attribute.

In [ ]:
# column names
flights.columns

## Selecting rows and columns

To select a particular column, we can index with its name. This can also be use to set values or create new columns.

In [ ]:
# select the carrier column
# you can also use flights.carrier
flights["carrier"]

In [ ]:
# create new column - delays other than departure delay
flights["other_delays"] = flights["arr_delay"]-flights["dep_delay"]
flights.head()

You can also access the individual columns as attributes. For example, `flights.carrier` will return the "carrier" column. Note that this syntax **cannot** be used to create new columns.

For selecting multiple columns, pass them as a list. For example, to select the "carrier" and "origin" columns:

In [ ]:
# selecting multiple columns
flights[["carrier","origin"]]

You can subset rows by regular numerical slices.

In [ ]:
# slicing the first five rows
flights[:5]

In [ ]:
# selecting the last 6 rows
flights[-6:] # negative indexing

You can combine row and column selection by doing each one succession in any order. For example, the following code 

1. first selects rows 23 to 29 (30 excluded)
2. then selects the carrier and origin column

The order can be reversed.

In [ ]:
# combining both row slicing and column selection
flights[23:30][["carrier","origin"]]

You can also subset rows and columns using traditional numerical slices with the `iloc` method. For example, the following code subsets rows 2 to 4 and columns 4 to 8.

In [ ]:
# Subsetting using numbers
flights.iloc[2:5,4:9]

## Logical subsets

In [ ]:
# subset departures from JFK
dat_JFK = flights[flights.origin=="JFK"]
dat_JFK

In [ ]:
# departues in either Los Angeles and San Francisco 
dat_cal = flights[flights.dest.isin(["LAX","SFO"])]
dat_cal

In [ ]:
# Example of numerical column bases subsetting
# Subset delayed arrivals
flights[flights.arr_delay>0]

In [ ]:
# multiple conditions can be combined using logical operators.
# Logical Operators in Pandas are &, | and ~ 
# subset departs from JFK to Los Angeles
flights[(flights.origin=="JFK") & (flights.dest=="LAX")]

You can combine logical subsets and selecting individual columns. The following code

1. selects flights which cover more than 2500 miles
2. then subsets the origin, dest, and distance columns
3. and finally, selects only unique rows.

In [ ]:
flights[flights.distance>2500][["origin","dest","distance"]].drop_duplicates()

Teh above `DataFrame` shows unique origin-destination pairs whose distance is greater than 2500 miles. These are direct flights from east-coast to cities in California (San Francisco, Sacremento, Oakland, San Jose), Alaska and Hawaii.

## Basic aggregation - counting

**Question**: What are the most common destinations from NYC airports?

The total number of flights to each destination can be found by using the `value_counts` method of the destination column. It returns the counts sorted in descending order. We can then select the top 10 destinations using regular row selections.

In [ ]:
flights.dest.value_counts()

In [ ]:
# select only the top 10 destinations and print a bar plot
top10 = flights.dest.value_counts()[:10]
top10.plot(kind="bar")
plt.ylabel("Total number of flights")
plt.show()

What about the carriers with the most number of flights? We can do it using the `value_counts` method of the carrier column.

In [ ]:
# Number of flights per carrier
flights.carrier.value_counts()

A more interesting question would be the number of flights for each carrier from each of the NYC airports. This can be done using the `crosstab` function.

In [ ]:
# Number of flights per carrier and origin airport
pd.crosstab(flights.carrier,flights.origin,margins=True)

## Groupby and aggregate

Suppose we want to analyze data for each of the three origin airports separately. One way would be to use logical subsets to create three new dataframes, one for each origin airport. 

The `groupby` method provides a convenient way to split the dataset by groups (in this case - groups in a single column) instead of manually creating subsets. It returns a `GroupBy` object. The different groups can then be iteratively accessed. This can be useful for doing similar analysis (eg: plots,logical subsets, etc.) on all the groups.

In [ ]:
origin_groups = flights.groupby(by=["origin"])
for group_name,group_dat in origin_groups:
    print("Origin Airport: {}".format(group_name))
    print(group_dat.head(5))
    print("*************")

A more useful feature of the `GroupBy` object is its `agg` or `aggregate` methods. These allow you to obtain summary statistics for some or all of the columns. The summary statistics for the columns can either be the same or different (particularly when they of different datatypes).

If the same summary statistic(s) is to be returned for all the groups, pass a list of functions/ function names (not always available). 

In [ ]:
origin_groups[["arr_delay","dep_delay"]].agg(['mean','median',np.std])

If different statistics are to be computed for different groups, pass a dictionary whose keys are the columns and their corresponding values are either a function or a list of functions (Names can be passed when available).

In [ ]:
dat_summary = origin_groups.agg({"arr_delay":['mean','median','std'],
                                "dep_delay":['min','median',np.max],
                                "distance":['mean'],
                                "dest":[pd.Series.nunique],
                                "origin":'count'})
dat_summary

To access the statistics for the individual columns, you can subset them like a regular DataFrame.

In [ ]:
# subset summary statistics of arr_delay
dat_summary["arr_delay"]

Groups need not be restricted to single columns. You can group by multiple columns. For example, 

In [ ]:
# groupby both origin and carrier
dat_multi_groups = flights.groupby(by=["origin","carrier"]).agg({"arr_delay":['mean','median','std']})
dat_multi_groups

## (Row) Index

For dataframes and series, index is the 'label' of the rows/observations. This can be used for identification and selection purposes. The index can be accessed by the `.index` method.

In [ ]:
# Index of flights dataset
flights.index

The index is not the same as the row number. For example,

In [ ]:
flights[flights.carrier=="OO"].index

In [ ]:
# Shuffle rows and print the first 10
flights.sample(frac=1,random_state=1)[:10]

To reset the index after performing the operation, use the `reset_index` method.

In [ ]:
# Shuffle rows, drop indices and print the first 10
flights.sample(frac=1,random_state=1).reset_index(drop=True).head(n=10)

To subset by index, you can use the `.loc` method. For example, to return the first few flights of `OO` carrier in 2014:

In [ ]:
flights.loc[flights[flights.carrier=="OO"].index[:10]]

Indexes are not restricted to being integer types - they can be of any type. For instance, the index of the top10 Series is of string type.

In [ ]:
print(top10.index)

If you want to replace this index with a regular RangeIndex and add the original index as the column, use the `reset_index` method with argument `drop=False` 

In [ ]:
top10.reset_index(drop=False,name="count")

The groupby aggregates created in the previous section have the group names as indices. For example:

In [ ]:
dat_summary.index

In [ ]:
# multi-level indices 
dat_multi_groups.index

Examples of accessing elements by index:

In [ ]:
# airports in New York City
dat_summary.loc[['LGA','JFK']]

In [ ]:
# UA and AA from JFK
dat_multi_groups.loc[[('JFK','UA'),('JFK','AA')]]